# Face Search: One-to-Many Face Matching

This notebook demonstrates how to build a face database and search for matching faces - useful for photo organization, security systems, and social media applications.